## **Dynamic-DataMasking**


#**_Column Level Security_**

In [0]:
select *
from sparksql_cata.sparksql_schema.orders_ext_table;
--Here in this below data ,[[ user_id ]] could be the PII(personally identifyable identity) that should not be revealed
-- to other team members , this sensitive data should be accessible only for you(admin).
-- To implement this data governence we need to understand about DataMasking.
-- What we will do:

--so we will create a function of DataMasking and implement that function on our data and then whenever someone will try to access the data , it will check authorization of that person if it will be admin data eill be accessed otherwise request will be rejected.

# **Create Mask Function**

In [0]:
create or replace function sparksql_cata.sparksql_schema.dynamic_mask(u_user_id STRING)
return
case when is_account_group_member('A_Admin') then u_user_id
ELSE "****--****" 
END;

# **Applying Dynamic Masking on Coulumn - user_id**

In [0]:
alter table sparksql_cata.sparksql_schema.orders_ext_table
alter column user_id set mask sparksql_cata.sparksql_schema.dynamic_mask;

-- Now anyone other than admin cannot access this coulumn
-- i am currently the admin of whole metastore so i will nit be able to test this.

# **_Row Level Security_**

In [0]:
select *
from sparksql_cata.sparksql_schema.orders_ext_table;

-- Row Level Security:
-- Lets suppose there are two teams A & B , Team_A needs to take care about the payment method credit/debit cards
-- and Team_B to paypal/UPI.  
-- here important thing is that A should not have access to B's data and vice versa.
-- To implement these type of security governence we need row level security.

In [0]:
-- 🔐 What is Row-Level Security (RLS)?
-- RLS controls which rows in a table a user or team can see, based on conditions like role, team, department, etc

# **Map Table**

In [0]:
-- First we will create the map table

create table sparksql_cata.sparksql_schema.map_table 
(
  payment_cateogory STRING,
  email STRING
)

In [0]:
INSERT INTO sparksql_cata.sparksql_schema.map_table
VALUES
      ('Credit Card' , 'humayun440@hotmail.com'),
      ('Debit Card','humayun440@hotmail.com'),
      ('PayPal','nomi5501@hotmail.com'),
      ('UPI','nomi5501@hotmail.com');

###**Map Table Testing**

In [0]:
select *
from sparksql_cata.sparksql_schema.map_table
where email = current_user()
and payment_cateogory = "Debit Card"

### **Converting Map Table into Boolean**

In [0]:
SELECT EXISTS
(
select *
from sparksql_cata.sparksql_schema.map_table
where email = current_user()
and payment_cateogory = 'Debit Card' 
)

### **Converting The Boolean into Function**

In [0]:
create or replace FUNCTION sparksql_cata.sparksql_schema.rowlevel_governence(p_payment_method STRING)
RETURNS BOOLEAN
LANGUAGE SQL
RETURN
(
   EXISTS
  (
    select *
    from sparksql_cata.sparksql_schema.map_table
    where email = current_user()
    and payment_cateogory = p_payment_method 
)

)

### **Applying the Function on Coulumn**

In [0]:
alter table sparksql_cata.sparksql_schema.orders_ext_table
SET ROW FILTER sparksql_cata.sparksql_schema.rowlevel_governence ON (payment_method) 

### **Testing_Final_Product**

In [0]:
select *
from sparksql_cata.sparksql_schema.orders_ext_table